<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Airline Tweets Sentiment Analysis Lab

_Authors: Phillippa Thomson (NYC)_

---

You are going to be analyzing tweets about airlines.  These have been hand-tagged with sentiment.  There are three categories: positive, neutral, and negative.

Use VADER to calculate sentiment for each tweet, and see if you can correctly predict the hand-tagged sentiment.

What is the accuracy?  Print out a heatmap to see where your model performs well, and where it performs poorly.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, \
precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tweets = pd.read_csv("../data/Tweets.csv", encoding = 'unicode_escape')

In [3]:
tweets

,airline_sentiment,airline,text
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.
1,positive,Virgin America,@VirginAmerica plus you've added commercials t...
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,negative,Virgin America,@VirginAmerica it's really aggressive to blast...
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing...
5,negative,Virgin America,@VirginAmerica seriously would pay $30 a fligh...
6,positive,Virgin America,"@VirginAmerica yes, nearly every time I fly VX..."
7,neutral,Virgin America,@VirginAmerica Really missed a prime opportuni...
8,positive,Virgin America,"@virginamerica Well, I didn'tÛ_but NOW I DO! :-D"
9,positive,Virgin America,"@VirginAmerica it was amazing, and arrived an ..."


### 1. Preview the airline_sentiment column.
- What percentage of reviews are positive, neutral, and negative?

In [4]:
!pip install vaderSentiment

In [5]:
# A:

### 2. Load in the Sentiment IntensityAnalyzer from Vader and add compound, negative, neutral, and positive scores into the DataFrame.

In [6]:
# A:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [7]:
# A:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score

In [8]:
# A:
sentiment_analyzer_scores("This airline is amazing")

{'neg': 0.0, 'neu': 0.441, 'pos': 0.559, 'compound': 0.5859}

In [9]:
sentiment_analyzer_scores("This airline sucks")

{'neg': 0.556, 'neu': 0.444, 'pos': 0.0, 'compound': -0.3612}

In [10]:
sentiment_analyzer_scores("This airline royally sucks")

{'neg': 0.455, 'neu': 0.545, 'pos': 0.0, 'compound': -0.3612}

In [30]:
tweets['test_col'] = tweets['text'].apply(sentiment_analyzer_scores)

In [34]:
new_df = tweets['test_col'].apply(pd.Series)

In [35]:
tweets = pd.concat([tweets,new_df], axis=1)

In [37]:
tweets.drop('test_col',axis=1,inplace=True)

In [38]:
tweets.head()

,airline_sentiment,airline,text,neg,neu,pos,compound
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.,0.000,1.000,0.0,0.0000
1,positive,Virgin America,@VirginAmerica plus you've added commercials t...,0.000,1.000,0.0,0.0000
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...,0.000,1.000,0.0,0.0000
3,negative,Virgin America,@VirginAmerica it's really aggressive to blast...,0.246,0.754,0.0,-0.5984
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing...,0.321,0.679,0.0,-0.5829


### 3. Store airline_sentiment in y to use as labels and create an appropriate feature matrix, X.

In [11]:
# A:
y = tweets['airline_sentiment']

In [12]:
tweets.head()

,airline_sentiment,airline,text
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.
1,positive,Virgin America,@VirginAmerica plus you've added commercials t...
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,negative,Virgin America,@VirginAmerica it's really aggressive to blast...
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing...


In [13]:
X = tweets['text']

In [15]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB         # Naive Bayes
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer

%matplotlib inline

In [16]:
# Split the new DataFrame into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [17]:
# Use CountVectorizer to create document-term matrices from X_train and X_test.
vect = CountVectorizer()
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

### 4. Fit a model of your choice to predict airline_sentient and cross-validate.

In [20]:
# Calculate null accuracy.
y_test_binary = np.where(y_test=='positive', 1, 0)

df.sentiment.loc[df.sentiment == 'positive'].count()
y_test_binary.mean()

0.16256830601092895

In [25]:
tweets.loc[tweets['airline_sentiment'] == 'positive'].count() / tweets['airline_sentiment'].count()

airline_sentiment    0.161407
airline              0.161407
text                 0.161407
dtype: float64

In [21]:
# A:
# Use Naive Bayes to predict the sentiment.
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

# Calculate accuracy of whether score is positive or negative
print((metrics.accuracy_score(y_test, y_pred_class)))

0.7631147540983606


In [ ]:
# A:

In [ ]:
# A:

In [ ]:
# A:

### 5. Display the confusion matrix.
- What reviews are difficult to identify?

In [ ]:
# A:

### 6. Print the classification report and discuss the characteristics of the model.

In [ ]:
# A:

The model does ok with negative tweets (the predominant class) but quite poorly with neutral.

To put this in perspective, human concordance, the probability that two people assign the same sentiment to an observation is usually around 70%-80% our baseline is at 63%. Even small increases in accuracy quickly move us towards a theoretical maximum in performance.